# Checkpoint Diversity Training
**Goal**: Train vanilla GRU h=64 seeds with epoch snapshots every 3 epochs (after epoch 8).
Each seed produces best + ~4 epoch checkpoints. Different training stages have different error
profiles, providing ensemble diversity that seed-only variation can't achieve.

**Plan**: 10 seeds x ~5 checkpoints = ~50 models. Flat ensemble of 10-15 diverse picks.

**Key insight**: Positive LB gap is driven by ensemble averaging/size, not val-optimization.
More decorrelated models >> fewer high-val models.

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup -- clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train 10 seeds with checkpoint diversity
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIG = "configs/gru_parity_v1_ckptdiv.yaml"
SEEDS = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

print(f"=== CHECKPOINT DIVERSITY TRAINING ===")
print(f"Config: {CONFIG}")
print(f"Seeds: {SEEDS}")
print(f"Epoch snapshots: every 3 epochs after epoch 8")
print("=" * 60, flush=True)

for seed in SEEDS:
    print(f"\n{'='*60}")
    print(f"Training seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    proc.wait()
    if proc.returncode != 0:
        print(f"ERROR: seed {seed} failed with rc={proc.returncode}")

print(f"\nAll {len(SEEDS)} seeds done!")

In [ ]:
# Cell 3: Evaluate all checkpoints (best + epoch)
import os, glob, torch
os.chdir("/content/competition_package")

GRU_BASE_MEAN = 0.2689  # mean of 23 GRU base seeds

results = []
for pt in sorted(glob.glob("logs/gru_parity_v1_ckptdiv_seed*.pt")):
    basename = os.path.basename(pt)
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "N/A")
    is_epoch = "_epoch" in basename
    tag = "epoch" if is_epoch else "best"
    # Extract seed
    seed_part = basename.split("seed")[1].split(".")[0].split("_")[0]
    results.append((seed_part, tag, epoch, score, basename))

results.sort(key=lambda x: -x[3])

print(f"{'Rank':<5} {'Seed':<6} {'Type':<6} {'Epoch':>6} {'Val Score':>10}")
print("-" * 42)
best_only = []
for i, (seed, tag, epoch, score, name) in enumerate(results, 1):
    marker = " *" if tag == "best" else ""
    print(f"{i:<5} s{seed:<5} {tag:<6} {str(epoch):>6} {score:>10.4f}{marker}")
    if tag == "best":
        best_only.append(score)

if best_only:
    mean_best = sum(best_only) / len(best_only)
    print(f"\nBest-only: {len(best_only)} seeds, mean={mean_best:.4f}, delta={mean_best - GRU_BASE_MEAN:+.4f}")
    all_scores = [s for _, _, _, s, _ in results]
    print(f"All ckpts: {len(results)} total, mean={sum(all_scores)/len(all_scores):.4f}")
    print(f"Top-10 of all: {sum(sorted(all_scores, reverse=True)[:10])/10:.4f}")

In [ ]:
# Cell 4: Strip ALL checkpoints (best + epoch) and zip
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

count = 0
for pt in sorted(glob.glob("logs/gru_parity_v1_ckptdiv_seed*.pt")):
    basename = os.path.basename(pt)
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    out = f"logs/slim/{basename}"
    torch.save(slim, out)
    orig = os.path.getsize(pt) / 1e6
    new = os.path.getsize(out) / 1e6
    print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")
    count += 1

print(f"\nStripped {count} checkpoints")

shutil.make_archive("/content/ckptdiv_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/ckptdiv_seeds.zip") / 1e6
print(f"\nckptdiv_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/ckptdiv_seeds.zip",
            "/content/drive/MyDrive/wunderfund/ckptdiv_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/ckptdiv_seeds.zip")